In [1]:
# BLOK 1 
# IMPORTOVANIE KNIZNIC

import pandas as pd
import numpy as np
import json

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, GRU, Concatenate
import tensorflow.keras.utils 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential

C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #

In [2]:
# BLOK 2 
# Nacitanie datasetu syntetickych kriviek

data_syn = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\df_all.csv_v2")

In [4]:
# BLOK 3
# Pridanie noveho atributu classes a classNum do povodneho datasetu a ich zmena na numpy pole. Atribut classNum bude pouzity ako
# cielovy atribut pri trenovani a vyhodnoteni modelu

classes = []
classNum = []
oc_S, dt_S, oc_noS, dt_noS = 0, 0, 0, 0

for index, row in data_syn.iterrows():
    if row["morphology"] == "over-contact" and row["spotty"] == 1: 
        classes.append("overSpotty")
        classNum.append(1)
        oc_S +=1
    elif row["morphology"] == "over-contact" and row["spotty"] == 0:
        classes.append("over")
        classNum.append(2)
        oc_noS +=1
    elif row["morphology"] == "detached" and row["spotty"] == 1: 
        classes.append("detSpotty")
        classNum.append(3)
        dt_S +=1
    elif row["morphology"] == "detached" and row["spotty"] == 0: 
        classes.append("det")
        classNum.append(4)
        dt_noS +=1
data_syn["classes"] = classes
data_syn["classNum"] = classNum
classes = np.array(classes)
classNum = np.array(classNum)
print('Over-contact spotty: ' + str(oc_S) + '\nDetached spotty: ' + str(dt_S) + '\nOver-contact no spotty: ' + str(oc_noS) + '\nDetached no spotty: ' + str(dt_noS))

Over-contact spotty: 45320
Detached spotty: 453662
Over-contact no spotty: 45869
Detached no spotty: 445556


In [5]:
# BLOK 4 
# Diskretizacia

newDataAll = []
for row in data_syn['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newDataAll.append(newRow)
newDataAll = np.array(newDataAll)

In [10]:
# BLOK 5
# Rozdelenie upravenych kriviek na trenovaciu a testovaciu mnozinu v pomere 80:20

print("Number of light curves: " + str(len(newDataAll)) + "\nTarget attribute: " + str(len(classNum)))
X_trainAll4, X_testAll4, y_trainAll4, y_testAll4 = train_test_split(newDataAll, classNum, test_size=0.2)
y_trainAll4 = to_categorical(y_trainAll4-1, 4)
y_testAll4 = to_categorical(y_testAll4-1, 4)
print("Length X_train: ", str(len(X_trainAll4)))
print("Length y_train: ", str(len(y_trainAll4)))
print("Length X_test: ", str(len(X_testAll4)))
print("Length y_test: ", str(len(y_testAll4)))

Number of light curves: 990407
Target attribute: 990407
Length X_train:  792325
Length y_train:  792325
Length X_test:  198082
Length y_test:  198082


In [ ]:
# BLOK 6
# Vytvorenie architektúry klasifikačneho modelu a jeho trenovanie

inputs = Input(shape=(100, 1))
a = LSTM(64, dropout = 0.1, recurrent_dropout = 0.1, return_sequences = True)(inputs)
a = Flatten()(a)
b = Convolution1D(64, 3, padding = "valid", input_shape = (100, 1))(inputs)
b = MaxPooling1D(pool_size=2)(b)
b = Convolution1D(32, 3, padding = "valid")(b)
b = MaxPooling1D(pool_size=2)(b)
b = Flatten()(b)
x = Concatenate()([a, b])
x = Dropout(0.5)(x)
x = Dense(64, activation = "relu")(x)
output = Dense(4, activation = "softmax")(x)
model4Class = Model(inputs = inputs, outputs = output)
model4Class.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["acc"])

saved_model = "new_model4Class.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor = "val_loss", verbose = 2, save_best_only = True, mode = 'min')

model4Class.fit(X_trainAll4, y_trainAll4, epochs = 5, batch_size = 32, validation_split=0.2, callbacks = [checkpoint], verbose = 2)

In [11]:
# BLOK 7
# Nacitanie modelu. Predikovanie syntetickych kriviek a vyhodnotenie modelu na testovacej mnnozine syntetickych dat pomocou 
# confusion matrix a vypocet presnosti, navratnosti a f1 skore

model = load_model('new_model4Class.hdf5')
y_predAll4 = model.predict(X_testAll4)
y_predAll42 = []
for i in y_predAll4:
    maximum = np.argmax(i)
    y_predAll42 = np.append(y_predAll42, maximum)

cm = confusion_matrix(y_testAll4.argmax(axis=1), y_predAll42)
print("Confusion Matrix 1D CNN + LSTM (4 class.): \n" + str(cm))

prfs = classification_report(y_testAll4.argmax(axis=1), y_predAll42)
print(str(prfs))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Confusion Matrix 1D CNN + LSTM (4 class.): 
[[ 2660  6321   117    13]
 [    8  9215    32     6]
 [   66    30 47935 42508]
 [   37    46   331 88757]]
              precision    recall  f1-score   support

           0       0.96      0.29      0.45      9111
           1       0.59      1.00      0.74      9261
           2       0.99      0.53      0.69     90539
           3       0.68      1.00      0.81     89171

    accur